# TODO: 
   - rechercher le significations des extensions pour avoir quelque chose de plus explicite 
       - hn1: neige fraiche 1 jour
       - hn3: neige fraiche 3 jours
       - hsr2000: hauteur de neige a 2000m
       - hrs2500: hauteur de neige a 2500m
       - hsrel: enneigement compare a la moyenne pluriannuelle
       - hstop: hauteur de neige
   - modifider dates dataframe pour martch avec les json 
   - function is in to check wether point belongs to selected polygon
   - iterate over polygons of a map to assign values to each accident
    

In this notebook we will go through each accident registered in our dataset and look at the snow informations retrieved from the maps dataset to relate both of them.

In [1]:
import numpy as np
import pandas as pd
import folium
import branca
import json
import seaborn as sns
import matplotlib.pyplot as plt
import os

%matplotlib inline

In [2]:
# Load avalanche accidents data
df_accidents = pd.read_csv('../data/accidents/raw_accidents.csv')
coordinates = pd.read_csv('../data/accidents/coordinates.csv')

In [3]:
df_accidents = df_accidents.assign(Longitude = coordinates.Longitude, Latitude = coordinates.Latitude)
df_accidents = df_accidents.drop(['Aspect', 'Activity', 'Danger level', 'caught', 'buried', 'killed'], axis=1)
#df_accidents['Date'] = pd.to_datetime(df_accidents['Date'])
df_accidents.head()

,Date,Canton,Starting zone,Elevation,Latitude,Longitude
0,26.10.97,BE,Kandersteg,3420,46.469813,7.728972
1,19.12.97,GR,St. Moritz,2560,46.502976,9.772840
2,21.12.97,UR,Andermatt,2940,46.602754,8.612660
3,29.12.97,VS,Bagnes,2435,46.034050,7.199696
4,04.01.98,UR,Andermatt,2400,46.617105,8.616889


Our main interest here are the places and dates.

In [4]:
df_accidents['Date'] = pd.to_datetime(df_accidents['Date'])
#changer datetime pour obtenir dd.mm.aaaa
df_accidents.head()

,Date,Canton,Starting zone,Elevation,Latitude,Longitude
0,1997-10-26,BE,Kandersteg,3420,46.469813,7.728972
1,1997-12-19,GR,St. Moritz,2560,46.502976,9.772840
2,1997-12-21,UR,Andermatt,2940,46.602754,8.612660
3,1997-12-29,VS,Bagnes,2435,46.034050,7.199696
4,1998-04-01,UR,Andermatt,2400,46.617105,8.616889


To each accident we will track the relevent snow information, i.e., from the map dataset we will retrieve the snow level at the location of the accident for the 10 preceding days and the following 3 days.

In [5]:
# admettons que cette ligne fonctionne et qu'il s'agisse d'un array avec tous les noms des json 
json_path = '../data/json-maps'
json_file_names = os.listdir(json_path)

extensions = ['hn1', 'hn3', 'hsr2000', 'hsr2500', 'hsrel', 'hstop']

json_hn1, json_hn3, json_hsr2000, json_hsr2500, json_hsrel, json_hstop = [], [], [], [], [], []
json_files = [json_hn1, json_hn3, json_hsr2000, json_hsr2500, json_hsrel, json_hstop]

for json_ in json_file_names:
    for i, ext in enumerate(extensions):
        if ext in json_:
            json_files[i].append(json.loads(open(os.path.join(json_path, json_)).read()))


In [6]:
df_accidents = pd.concat([df_accidents, pd.DataFrame(columns=extensions)])

In [64]:
def surround_days(df, json_snow):
    h = np.empty_like(len(df['Date']), dtype=None)
    for i, dat in enumerate(df_accidents['Date']):
        for j, json_ in enumerate(json_snow):
            d = str(dat.day) + "." + str(dat.month) + "." + str(dat.year)
            if(d == json_.get("date")):
                h[i].append(json_snow[j])
    return h
                
hn1 = surround_days(df_accidents, json_hn1)
hn3 = surround_days(df_accidents, json_hn3)
hsr2000 = surround_days(df_accidents, json_hsr2000)
hsr2500 = surround_days(df_accidents, json_hsr2500)
hsrel = surround_days(df_accidents, json_hsrel)
hstop = surround_days(df_accidents, json_hstop)

In [65]:
len(hn1)

TypeError: len() of unsized object

In [ ]:
#apply is_in to each element and iterate over all polygons 
df_accidents.loc[:, 'hn1'] = hn1
df_accidents.loc[:, 'hn3'] = hn3
df_accidents.loc[:, 'hsr2000'] = hsr2000
df_accidents.loc[:, 'hsr2500'] = hsr2500
df_accidents.loc[:, 'hsrel'] = hsrel
df_accidents.loc[:, 'hstop'] = hstop

In [ ]:
df_accidents.tail(50)

In [ ]:
for i, values in df_accidents.iterrows():
    
    information = [
        ("Date", values["Date"].strftime("%d.%m.%Y")),
        ("Starting zone", values["Starting zone"]),
        ("Elevation", values["Elevation"]),
        ("Snow hn1", values['hn1']),
        ("Snow hn3", values['hn3']),
        ("Snow hsr2000", values['hsr2000']),
        ("Snow hsr2500", values['hsr2500']),
        ("Snow hsrel", values['hsrel']),
        ("Snow hstop", values['hstop']),
    ]